In [1]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

In [ ]:
video_path = 'videos_ashih/ashih2.mp4' 

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Video not opened")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break 

    cv2.imshow("Video", frame)

    # close the window by pressing q
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# free ressources
cap.release()
cv2.destroyAllWindows()
